# TODO:
- ~~enforce output format for gemini~~
- llama, gpt, claude
- add evaluation if there is a golden set for individual model + compare result with other LLMs
- aggregation strategy
- add for multi-class/ner
- add images


nice things to do:
- add tqdm to asyncio calls
- proper logging

In [3]:
from utils import Annotate
from datasets import load_dataset

seed =42

/usr/local/google/home/amirimani/Desktop/projects/LLMTest_NeedleInAHaystack/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
gemini_prompt_template = """
<data_description>
{description}
</data_description>
-----------

<context>
{datapoint}
</context>
------------

<labels>
{labels}
</labels>
------------

INSTRUCTION:
- familirize yourself with the data using data_description
- read the context carefully. this is the data point you need to label.
- take your time and label the dadatapoint with the most appropriate option using the provided labels.
- return the result as a single label from the <labels>. Don't provide explanations
"""

In [5]:
dataset = load_dataset("yelp_polarity", split="train") # https://huggingface.co/datasets/yelp_polarity

# take a small sample for dev purposes
dataset_sample = dataset.shuffle(seed=seed).select(range(1000))

# user provided data description
DESCRIPTION = """
This is a dataset for binary sentiment classification.
It contains highly polar yelp reviews.
Negative polarity is class 0, and positive class 1.
"""

LABEL_SET = [0, 1] 

In [6]:
prompt = [gemini_prompt_template.format(description= DESCRIPTION,
                                        datapoint=x,
                                        labels=LABEL_SET) for x in dataset_sample["text"][:5]]
len(prompt)

5

In [7]:
ann = Annotate()

In [8]:
result = await ann.TextClassification(prompt, model="gemini")

[1, 0, 0, 1, 1]


In [9]:
dataset_sample["label"][:5]

[1, 0, 0, 1, 1]